In [ ]:
'''
L'objectif de ce projet est d'observer s'il existe des 'déserts sportifs', 
lieux en France où les infrastrcutures sportives manquent. 
La question sera alors d'essayer d'expliquer ces déserts sportifs, que ce soit par des raisons économiques ou politiques. 
Enfin, il s'agira de comparer la carte des déserts sportifs avec d'autres cartes connues, à l'instar des déserts médicaux.
'''

In [8]:
''' 
Il convient d'abord d'importer tous les modules python nécessaires au travail
'''

import numpy as np
import numpy.linalg as al
import matplotlib.pyplot as plt 
import pandas as pd
import geopandas as gpd
from cartiflette import carti_download
import requests
import zipfile
import io

In [9]:
'''
On importe ensuite nos jeux de données
Le jeu de données principal
'''
url = "https://data.sports.gouv.fr/api/explore/v2.1/catalog/datasets/equipements-sportifs/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
equipement = pd.read_csv(url, sep=";", low_memory=False)

'''
Un jeu de données csv sur des informations économiques et démographiques au niveau des communes
'''

urlpop = "https://www.insee.fr/fr/statistiques/fichier/2521169/base_cc_comparateur_csv.zip"
response = requests.get(urlpop)

with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    with z.open("base_cc_comparateur.csv") as csvfile:
        df_communes = pd.read_csv(csvfile, sep=";", low_memory=False)

'''
Deux jeux de données politiques au niveau des communes: résultats des législatives 2024 des 1er et 2nd tours
'''

url1="https://www.data.gouv.fr/api/1/datasets/r/bd32fcd3-53df-47ac-bf1d-8d8003fe23a1"
url2="https://www.data.gouv.fr/api/1/datasets/r/5a8088fd-8168-402a-9f40-c48daab88cd1"

legislatives1=pd.read_csv(url1, sep=";", low_memory=False)
legislatives2=pd.read_csv(url2, sep=";", low_memory=False)


In [52]:
'''
Il s'agit maintenant de laver nos jeux de données afin de les réunir ensuite à l'aide des codes INSEE
On conserve seulement les variables qui peuvent nous intéresser 
'''

cols = ["Nom de l'installation sportive", "Code Postal", "Commune nom", "Commune INSEE", "Département Code", "Département Nom", "Densite Catégorie", "Nom de l'équipement sportif", "Type d'équipement sportif", "Longitude", "Latitude"]
equipement = equipement[cols]

cols=["CODGEO", "P22_POP", "NAISD24", "DECESD24", "P22_MEN","MED21", "TP6021", "P22_CHOM1564"]
df_communes = df_communes[cols]


cols=["Code commune", "Libellé commune"]
cols= cols + [(f"% Voix/exprimés {i}") for i in range(1,9)]+[ (f"Nuance candidat {i}") for i in range(1, 9)]
legislatives1= legislatives1[cols]

cols=["Code commune", "Libellé commune"]
cols= cols + [(f"% Voix/exprimés {i}") for i in range(1,4)]+[ (f"Nuance candidat {i}") for i in range(1, 4)]
legislatives2= legislatives1[cols]

      Code commune          Libellé commune % Voix/exprimés 1  \
0             1001  L'Abergement-Clémenciat            17,44%   
1             1002    L'Abergement-de-Varey             1,17%   
2             1004        Ambérieu-en-Bugey             0,87%   
3             1005      Ambérieux-en-Dombes            18,38%   
4             1006                  Ambléon            33,85%   
...            ...                      ...               ...   
35227        ZZ235         Bahamas (Nassau)             4,00%   
35228        ZZ236                   Astana             0,00%   
35229        ZZ237                  Mossoul             0.00%   
35230        ZZ238                 Florence             2,34%   
35231        ZZ239                  Managua             5,81%   

      % Voix/exprimés 2 % Voix/exprimés 3 % Voix/exprimés 4 % Voix/exprimés 5  \
0                 0,00%             0,00%             0,42%            18,07%   
1                 0,58%             2,92%            36,2